# CSE 151A Group Project

In [1]:
import pandas as pd
import datetime as dt

## Data Exploration

Start by loading in the dataset. Since the reviews csv doesn't provide anything aside from the fact that a certain location id got reviewed by a certain user id, we opted to not use it. 

In [2]:
listings_df = pd.read_csv('Listings.csv', encoding='ISO-8859-1')
listings_df.head()

C:\Users\mybri\AppData\Local\Temp\ipykernel_33304\1526227757.py:1: DtypeWarning: Columns (5,13) have mixed types. Specify dtype option on import or set low_memory=False.
  listings_df = pd.read_csv('Listings.csv', encoding='ISO-8859-1')


,listing_id,name,host_id,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,...,minimum_nights,maximum_nights,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable
0,281420,"Beautiful Flat in le Village Montmartre, Paris",1466919,2011-12-03,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
1,3705183,39 mÃÂ² Paris (Sacre CÃ âur),10328771,2013-11-29,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
2,4082273,"Lovely apartment with Terrace, 60m2",19252768,2014-07-31,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
3,4797344,Cosy studio (close to Eiffel tower),10668311,2013-12-17,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
4,4823489,Close to Eiffel Tower - Beautiful flat : 2 rooms,24837558,2014-12-14,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f


Let's start by looking at the shape of the dataset.

In [3]:
listings_df.shape

(279712, 33)

We have 279712 locations, each with 33 variables associated with them. 

Let's start by taking a look at what each column means.

In [4]:
listings_info = pd.read_csv('Listings_data_dictionary.csv', encoding='ISO-8859-1')
listings_info

,Field,Description
0,listing_id,Listing ID
1,name,Listing Name
2,host_id,Host ID
3,host_since,Date the Host joined Airbnb
4,host_location,Location where the Host is based
5,host_response_time,Estimate of how long the Host takes to respond
6,host_response_rate,Percentage of times the Host responds
7,host_acceptance_rate,Percentage of times the Host accepts a booking...
8,host_is_superhost,Binary field to determine if the Host is a Sup...
9,host_total_listings_count,Total listings the Host has in Airbnb


Now that we have a look at what each column means, let's start by dropping every column that couldn't possibly be used to predict the rating of the listing.

We'll also drop any columns that are associated with the rating, because that information is not available at the time of booking.

In [5]:
listings_df.columns

Index(['listing_id', 'name', 'host_id', 'host_since', 'host_location',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_total_listings_count',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'district', 'city', 'latitude', 'longitude', 'property_type',
       'room_type', 'accommodates', 'bedrooms', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'instant_bookable'],
      dtype='object')

In [6]:
unusable_cols = ["listing_id","name","host_id"]
unavailable_cols = ['review_scores_accuracy', 'review_scores_cleanliness',
                    'review_scores_checkin', 'review_scores_communication',
                    'review_scores_location', 'review_scores_value']

listings_df = listings_df.drop(unusable_cols,axis=1)
listings_df = listings_df.drop(unavailable_cols,axis=1)
listings_df.columns

Index(['host_since', 'host_location', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_total_listings_count', 'host_has_profile_pic',
       'host_identity_verified', 'neighbourhood', 'district', 'city',
       'latitude', 'longitude', 'property_type', 'room_type', 'accommodates',
       'bedrooms', 'amenities', 'price', 'minimum_nights', 'maximum_nights',
       'review_scores_rating', 'instant_bookable'],
      dtype='object')

Next, we're going to take a look at the missing values in the dataset.

In [7]:
for column in listings_df.columns:
    print(column + ": " + str(listings_df[column].isna().sum()) + " missing values")

host_since: 165 missing values
host_location: 840 missing values
host_response_time: 128782 missing values
host_response_rate: 128782 missing values
host_acceptance_rate: 113087 missing values
host_is_superhost: 165 missing values
host_total_listings_count: 165 missing values
host_has_profile_pic: 165 missing values
host_identity_verified: 165 missing values
neighbourhood: 0 missing values
district: 242700 missing values
city: 0 missing values
latitude: 0 missing values
longitude: 0 missing values
property_type: 0 missing values
room_type: 0 missing values
accommodates: 0 missing values
bedrooms: 29435 missing values
amenities: 0 missing values
price: 0 missing values
minimum_nights: 0 missing values
maximum_nights: 0 missing values
review_scores_rating: 91405 missing values
instant_bookable: 0 missing values


The first thing we noticed was that many of the host columns had the same number of missing values, which we guessed meant that AirBNB simply didn't have information on those hosts. Let's verify this.

In [8]:
missing_165 = ['host_since', 'host_is_superhost', 'host_total_listings_count', "host_has_profile_pic", "host_identity_verified"]
missing_count = listings_df[listings_df[missing_165].isna().all(axis=1)].shape[0]

print(missing_count == 165)

True


Great. We're going to delete all rows that have missing data in all of these columns, as we think that sacrificing such a small fraction of the dataset for additional variables is worthwhile.

In [9]:
listings_df = listings_df[listings_df[missing_165].isna().all(axis=1) == False]
listings_df.shape

(279547, 24)

We also decided that due to the amount of missing values in host response times, host response rates, district, and host acceptance rates, we should omit those variables.

In [10]:
listings_df = listings_df.drop(columns=['host_response_time','host_response_rate', 'host_acceptance_rate', "district"])

Bedrooms was a tough decision given the amount of missing data, but we opted to drop all rows where the number of bedrooms was missing, considering that it'd be tough to make a decision on booking an AirBNB without its number of bedrooms. More on this later.

In [11]:
listings_df = listings_df[listings_df["bedrooms"].isna() == False]

We decided latitude and longitude would be redundant when city is already provided, so we dropped that variable. We'll also drop neighborhood, as we feel it's unlikely to have enough data points for each neighborhood to be of use.

In [12]:
listings_df = listings_df.drop(columns=["latitude","longitude", "neighbourhood"])

We'll also drop host_location, as we don't see it being effective in predicting rating.

In [13]:
listings_df = listings_df.drop(columns=["host_location"])

Finally, we're also going to have to drop all rows where review_scores_rating is null, as this is the variable we need to predict.

In [14]:
listings_df = listings_df[listings_df["review_scores_rating"].isna() == False]

Great, now we have eliminated all of our missing values.

In [15]:
for column in listings_df.columns:
    print(column + ": " + str(listings_df[column].isna().sum()) + " missing values")

host_since: 0 missing values
host_is_superhost: 0 missing values
host_total_listings_count: 0 missing values
host_has_profile_pic: 0 missing values
host_identity_verified: 0 missing values
city: 0 missing values
property_type: 0 missing values
room_type: 0 missing values
accommodates: 0 missing values
bedrooms: 0 missing values
amenities: 0 missing values
price: 0 missing values
minimum_nights: 0 missing values
maximum_nights: 0 missing values
review_scores_rating: 0 missing values
instant_bookable: 0 missing values


In [16]:
listings_df.shape

(168414, 16)

We're now down to 168414 entries with 17 variables. Let's walk through the design rationale for each column:

- host_since -- we want the amount of time a host has been active, as we think this would've given them time to adjust their properties to be of higher quality.
- All other host variables -- gives us a decent idea of the quality of the host.
- city -- gives us a good idea of their rough location.
- property_type and room_type -- the type of housing may affect how well they're generally rated
- accomodates / bedrooms -- the size of the AirBNB might follow general trends in rating
- amenities -- gives us the added benefits, which may add a convenience that'll boost rating.
- price -- rating may be correlated with pricing, one way or another
- minimum_nights and maximum_nights -- we think that this'll add to or subtract from the convenience for the user, leading to correlation with rating
- instant_bookable -- same as above.

Now, let's begin by evaluating for preprocessing.

In [17]:
listings_df.head()

,host_since,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,city,property_type,room_type,accommodates,bedrooms,amenities,price,minimum_nights,maximum_nights,review_scores_rating,instant_bookable
0,2011-12-03,f,1.0,t,f,Paris,Entire apartment,Entire place,2,1.0,"[""Heating"", ""Kitchen"", ""Washer"", ""Wifi"", ""Long...",53,2,1125,100.0,f
1,2013-11-29,f,1.0,t,t,Paris,Entire apartment,Entire place,2,1.0,"[""Shampoo"", ""Heating"", ""Kitchen"", ""Essentials""...",120,2,1125,100.0,f
2,2014-07-31,f,1.0,t,f,Paris,Entire apartment,Entire place,2,1.0,"[""Heating"", ""TV"", ""Kitchen"", ""Washer"", ""Wifi"",...",89,2,1125,100.0,f
3,2013-12-17,f,1.0,t,t,Paris,Entire apartment,Entire place,2,1.0,"[""Heating"", ""TV"", ""Kitchen"", ""Wifi"", ""Long ter...",58,2,1125,100.0,f
4,2014-12-14,f,1.0,t,f,Paris,Entire apartment,Entire place,2,1.0,"[""Heating"", ""TV"", ""Kitchen"", ""Essentials"", ""Ha...",60,2,1125,100.0,f


First, since we want host_since to represent the amount of time the host has been active, we'll convert from datetime to a straight integer by subtracting it from the current date.

In [18]:
listings_df["host_since"] = pd.to_datetime(listings_df["host_since"])
listings_df["host_since"] = listings_df["host_since"].apply(lambda x: dt.datetime.now() - x)
listings_df["host_since"] = listings_df["host_since"].apply(lambda x: x.days)

Next, we'll convert all columns consisting of t's and f's to 1's and 0's such that it can be more easily used by ML models.

In [19]:
tf_cols = ["host_is_superhost", "host_has_profile_pic", "host_identity_verified", "instant_bookable"]
listings_df[tf_cols] = listings_df[tf_cols].replace({'t': 1, 'f': 0})

Next, we'll one-hot encode all of the categorical variables with the exception of amenities.

In [20]:
cat_cols = ["property_type", "room_type", "city"]
listings_df = pd.get_dummies(listings_df, columns=cat_cols, dtype=int)

Then we will find all of the different amenities offered and choose the top 100 to check for existence in provided amenities

In [21]:
amenities = {}

for item in listings_df['amenities']:
    # Separate all all of the amenities for the row
    amens = item.strip('[]').split(',')
    for idx in range(len(amens)):
        # Increase the count of the amenity in the dictionary
        amens[idx] = amens[idx].strip(' "')
        amenities[amens[idx]] = amenities.get(amens[idx],0) + 1
# Find the top 100 amenities
result = list(amenities.items())
result.sort(reverse=True, key=lambda x: x[1])
result[:100]


[('Wifi', 160938),
 ('Essentials', 157852),
 ('Kitchen', 148161),
 ('Long term stays allowed', 146193),
 ('Hangers', 135859),
 ('TV', 130859),
 ('Hair dryer', 124085),
 ('Iron', 123092),
 ('Hot water', 117817),
 ('Dedicated workspace', 117344),
 ('Heating', 115566),
 ('Washer', 114897),
 ('Shampoo', 113683),
 ('Smoke alarm', 99861),
 ('Dishes and silverware', 96346),
 ('Refrigerator', 94820),
 ('Air conditioning', 87001),
 ('Cooking basics', 85488),
 ('Microwave', 77364),
 ('Bed linens', 74422),
 ('Stove', 73215),
 ('Coffee maker', 69705),
 ('Oven', 65895),
 ('Elevator', 63744),
 ('Fire extinguisher', 62999),
 ('Dryer', 58849),
 ('First aid kit', 55427),
 ('Carbon monoxide alarm', 53918),
 ('Extra pillows and blankets', 48608),
 ('Free street parking', 47041),
 ('Private entrance', 44749),
 ('Cable TV', 44138),
 ('Luggage dropoff allowed', 43332),
 ('Free parking on premises', 42819),
 ('Host greets you', 40563),
 ('Patio or balcony', 40189),
 ('Dishwasher', 35759),
 ('Lock on bedroom 

In [22]:
# Add them each into the dataframe
amenities_df = pd.DataFrame()
for item in result[:100]:
    amenities_df[item[0]] = listings_df['amenities'].str.contains(item[0]).astype(int)
listings_df = pd.concat([listings_df,amenities_df],axis=1)

In [23]:
listings_df

,host_since,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,accommodates,bedrooms,amenities,price,minimum_nights,...,Bathroom essentials,Baby safety gates,Bread maker,Clothing storage,Full kitchen,Fireplace guards,Sound system,EV charger,Outdoor furniture,Lake access
0,4451,0,1.0,1,0,2,1.0,"[""Heating"", ""Kitchen"", ""Washer"", ""Wifi"", ""Long...",53,2,...,0,0,0,0,0,0,0,0,0,0
1,3724,0,1.0,1,1,2,1.0,"[""Shampoo"", ""Heating"", ""Kitchen"", ""Essentials""...",120,2,...,0,0,0,0,0,0,0,0,0,0
2,3480,0,1.0,1,0,2,1.0,"[""Heating"", ""TV"", ""Kitchen"", ""Washer"", ""Wifi"",...",89,2,...,0,0,0,0,0,0,0,0,0,0
3,3706,0,1.0,1,1,2,1.0,"[""Heating"", ""TV"", ""Kitchen"", ""Wifi"", ""Long ter...",58,2,...,0,0,0,0,0,0,0,0,0,0
4,3344,0,1.0,1,0,2,1.0,"[""Heating"", ""TV"", ""Kitchen"", ""Essentials"", ""Ha...",60,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279707,3224,0,1.0,1,1,2,1.0,"[""Iron"", ""Heating"", ""Washer"", ""Dedicated works...",120,1,...,0,0,0,0,0,0,0,0,0,0
279708,3726,0,1.0,1,1,2,1.0,"[""Shampoo"", ""Iron"", ""Heating"", ""Washer"", ""Hair...",60,7,...,0,0,0,0,0,0,0,0,0,0
279709,4305,0,1.0,1,1,2,1.0,"[""Paid parking off premises"", ""Shampoo"", ""Firs...",50,6,...,0,0,0,0,0,0,0,0,0,0
279710,3130,0,1.0,1,1,2,1.0,"[""TV"", ""Iron"", ""Kitchen"", ""Hangers"", ""Smoke al...",105,3,...,0,0,0,0,0,0,0,0,0,0


Now let's separate the data we will use as input (still needs to be normalized) and also get the output data

In [29]:
# Get input dataframe
input_df = listings_df.iloc[:,(listings_df.columns != 'amenities') & (listings_df.columns != 'review_scores_rating')]
input_df.describe()

,host_since,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,accommodates,bedrooms,price,minimum_nights,maximum_nights,...,Bathroom essentials,Baby safety gates,Bread maker,Clothing storage,Full kitchen,Fireplace guards,Sound system,EV charger,Outdoor furniture,Lake access
count,168414.000000,168414.000000,168414.000000,168414.000000,168414.000000,168414.000000,168414.000000,168414.000000,168414.000000,1.684140e+05,...,168414.000000,168414.000000,168414.000000,168414.000000,168414.000000,168414.000000,168414.000000,168414.000000,168414.000000,168414.000000
mean,2970.914502,0.247462,9.384867,0.998272,0.780351,3.456524,1.497411,498.078414,7.068973,3.243060e+04,...,0.010557,0.010207,0.009500,0.016252,0.008301,0.007339,0.007499,0.005997,0.005997,0.005409
std,864.693864,0.431538,57.511888,0.041532,0.414010,2.133170,1.033571,2156.167223,33.383495,7.791564e+06,...,0.102205,0.100513,0.097006,0.126442,0.090731,0.085354,0.086274,0.077209,0.077209,0.073349
min,1084.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,8.000000,1.000000,1.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2307.000000,0.000000,1.000000,1.000000,1.000000,2.000000,1.000000,75.000000,1.000000,4.500000e+01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3008.000000,0.000000,1.000000,1.000000,1.000000,3.000000,1.000000,145.000000,2.000000,1.125000e+03,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3586.000000,0.000000,4.000000,1.000000,1.000000,4.000000,2.000000,421.000000,4.000000,1.125000e+03,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5659.000000,1.000000,7235.000000,1.000000,1.000000,16.000000,50.000000,300177.000000,9999.000000,2.147484e+09,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [28]:
# Get output dataframe
output_df = listings_df['review_scores_rating']
output_df.describe()

count    168414.000000
mean         93.577149
std           9.956950
min          20.000000
25%          92.000000
50%          96.000000
75%         100.000000
max         100.000000
Name: review_scores_rating, dtype: float64